In [2]:
import os
import time  # for adding a delay between retries
from tqdm import tqdm
import json
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

# Number of retries
max_retries = 3
retry_delay = 5  # seconds between retries

In [3]:
# Utility functions
def get_gpt_response_with_retries(prompt_obj, client, model, retries=max_retries):
    for attempt in range(retries):
        try:
            response = client.completions.create(
                model=model,
                prompt=prompt_obj['prompt'],
                max_tokens=200,
                temperature=0.7
            )
            prompt_obj['response'] = response.choices[0].text.strip()
            return  prompt_obj # Extract and clean response
        except Exception as e:
            print(f"Error on attempt {attempt + 1}/{retries}: {e}")
            if attempt < retries - 1:
                time.sleep(retry_delay)  # Wait before retrying
            else:
                raise Exception(f"Failed after {retries} attempts: {e}")

# rewrite the case to have the prompt template format
def rewrite_case_prompt(case):

    try:
        client = OpenAI(
            api_key=os.environ.get('CBORG_EXT_API_KEY_FLASH'),
            base_url="https://api.cborg.lbl.gov/v1"
        )

        response = client.chat.completions.create(
            model='google/gemini-flash',
            messages=[
                {
                    'role': 'user',
                    'content': f"""Rewrite the following medical case into this template differnt : "Patient Information: \nAge: [age]\nGender: [gender]\nDisease Onset:[disease_onset]\nChief Complaint: [complaints]", where the values in square brackets [] are derived from the case or "Unknown" if not specified. Please remove square brackets
    **Medical Case:**
    {case}
    """
                }
            ],
            temperature=0.0
        )
        return response.choices[0].message.content.rstrip() + "\n\nPlease provide a detailed and comprehensive diagnostic analysis of this medical record.\n\nAssistant:"
    except Exception as e:
        print("ERROR", e)

# Function to get gold standard with prompts
def get_prompts(file_path):
    all_prompts = []
    # Open the JSONL file
    with open(file_path, 'r') as file:
        # Read each line in the file
        for line in file:
            all_prompts.append(json.loads(line))
    return all_prompts

# Function to process a single prompt from list of jsonl
def prompts_by_id(prompts, id):
    for prompt in prompts:
        if prompt['id'] == id:
            return prompt
    return None


In [18]:
# This code block creates a derivative medfound prompts from our exomiser-gold.jsonl prompts
def process_file(prompt_obj, lock):
    raw_prompt = prompt_obj['prompt']
    raw_prompt = raw_prompt.split("Here is the case:")[1]
    prompt_obj["prompt"] = rewrite_case_prompt(raw_prompt)
    with lock:
        with open("../data/medfound-prompts-test.jsonl", "a") as result_file:
            result_file.write(json.dumps(prompt_obj) + "\n")
    return True

def create_medfound_prompts(run=False):
    if run:
        with open(os.path.expanduser('~/CBORG_EXT_API_KEY_FLASH.txt'), 'r') as f:
            os.environ['CBORG_EXT_API_KEY_FLASH'] = f.read().strip()
        all_prompts = get_prompts("../data/exomiser-gold.jsonl")
        lock = threading.Lock()
        with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
            futures = {executor.submit(process_file, line, lock): line for line in all_prompts[:1]}
            for future in tqdm(as_completed(futures), total=len(futures)):
                future.result()
    else:
        print("Didn't run cause you probably didn't want to anyways.")
create_medfound_prompts()

Didn't run cause you probably didn't want to anyways.


In [4]:
# Run all inference on medfound prompts
def inference_medfound(model="medicalai/MedFound-176B"):
    with open(os.path.expanduser('~/CBORG_EXT_API_KEY.txt'), 'r') as f:
        os.environ['CBORG_EXT_API_KEY'] = f.read().strip()
    # Initialize the new API client
    client = OpenAI(
        api_key=os.environ.get("CBORG_EXT_API_KEY"),
        base_url="https://ext1.cborg.lbl.gov/v1"
    )
    all_prompts = get_prompts("../data/medfound-prompts.jsonl")
    already_processed = get_prompts("../data/results/medfound175b_results.jsonl")
    already_processed_ids = {prompt['id'] for prompt in already_processed}
    for p in all_prompts:
        p['prompt'] = p['prompt'].replace("None", "Unknown")
    lock = threading.Lock()
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = {executor.submit(get_gpt_response_with_retries, prompt_obj, client, model): prompt_obj for prompt_obj in all_prompts if prompt_obj['id'] not in already_processed_ids}
        for future in tqdm(as_completed(futures), total=len(futures)):
            with lock:
                prompt_obj = future.result()
                with open("../data/results/medfound175b_results.jsonl", "a") as result_file:
                    result_file.write(json.dumps(prompt_obj) + "\n")
inference_medfound()

100%|██████████| 1971/1971 [25:28<00:00,  1.29it/s]


In [8]:
def create_gemini_prompts():
    all_prompts = get_prompts("../data/exomiser-gold.jsonl")
    for prompt in all_prompts:
        prompt['prompt'] = prompt['prompt'] + "\n\n Do not explain the list, just provide the list of differential diagnoses."
    with open("../data/gemini-prompts.jsonl", "w") as result_file:
        for prompt in all_prompts:
            result_file.write(json.dumps(prompt) + "\n")
create_gemini_prompts()

In [11]:
# Run all inference on medfound prompts
def inference_gemini(model="google/gemini-flash"):
    with open(os.path.expanduser('~/CBORG_EXT_API_KEY_FLASH.txt'), 'r') as f:
        os.environ['CBORG_EXT_API_KEY_FLASH'] = f.read().strip()
    # Initialize the new API client
    client = OpenAI(
        api_key=os.environ.get("CBORG_EXT_API_KEY_FLASH"),
        base_url="https://api.cborg.lbl.gov/v1"
    )
    all_prompts = get_prompts("../data/gemini-prompts.jsonl")
    lock = threading.Lock()
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = {executor.submit(get_gpt_response_with_retries, prompt_obj, client, model): prompt_obj for prompt_obj in all_prompts}
        for future in tqdm(as_completed(futures), total=len(futures)):
            with lock:
                prompt_obj = future.result()
                with open("../data/results/geminiflash_results.jsonl", "a") as result_file:
                    result_file.write(json.dumps(prompt_obj) + "\n")
inference_gemini()

  5%|▍         | 256/5213 [00:35<29:18,  2.82it/s]

Error on attempt 1/3: Error code: 429 - {'error': {'message': '429: LiteLLM Rate Limit Handler for rate limit type = team. Crossed TPM / RPM / Max Parallel Request Limit. current rpm: 100, rpm limit: 100, current tpm: 0, tpm limit: 1000000, current max_parallel_requests: 0, max_parallel_requests: 9223372036854775807', 'type': 'None', 'param': 'None', 'code': '429'}}
Error on attempt 1/3: Error code: 429 - {'error': {'message': '429: LiteLLM Rate Limit Handler for rate limit type = team. Crossed TPM / RPM / Max Parallel Request Limit. current rpm: 100, rpm limit: 100, current tpm: 0, tpm limit: 1000000, current max_parallel_requests: 0, max_parallel_requests: 9223372036854775807', 'type': 'None', 'param': 'None', 'code': '429'}}
Error on attempt 1/3: Error code: 429 - {'error': {'message': '429: LiteLLM Rate Limit Handler for rate limit type = team. Crossed TPM / RPM / Max Parallel Request Limit. current rpm: 100, rpm limit: 100, current tpm: 0, tpm limit: 1000000, current max_parallel_

 18%|█▊        | 946/5213 [02:13<11:13,  6.34it/s]

Error on attempt 1/3: Error code: 429 - {'error': {'message': '429: LiteLLM Rate Limit Handler for rate limit type = team. Crossed TPM / RPM / Max Parallel Request Limit. current rpm: 100, rpm limit: 100, current tpm: 0, tpm limit: 1000000, current max_parallel_requests: 0, max_parallel_requests: 9223372036854775807', 'type': 'None', 'param': 'None', 'code': '429'}}


 19%|█▊        | 967/5213 [02:19<28:11,  2.51it/s]

...

 97%|█████████▋| 5052/5213 [12:39<00:57,  2.78it/s]

Error on attempt 1/3: Error code: 429 - {'error': {'message': '429: LiteLLM Rate Limit Handler for rate limit type = team. Crossed TPM / RPM / Max Parallel Request Limit. current rpm: 100, rpm limit: 100, current tpm: 0, tpm limit: 1000000, current max_parallel_requests: 0, max_parallel_requests: 9223372036854775807', 'type': 'None', 'param': 'None', 'code': '429'}}


 97%|█████████▋| 5064/5213 [12:44<01:00,  2.47it/s]

Error on attempt 1/3: Error code: 429 - {'error': {'message': '429: LiteLLM Rate Limit Handler for rate limit type = team. Crossed TPM / RPM / Max Parallel Request Limit. current rpm: 100, rpm limit: 100, current tpm: 0, tpm limit: 1000000, current max_parallel_requests: 0, max_parallel_requests: 9223372036854775807', 'type': 'None', 'param': 'None', 'code': '429'}}


100%|██████████| 5213/5213 [13:06<00:00,  6.63it/s]
